In [1]:
# notebook setup
import datetime as dt
import os
import sys

from IPython.core.interactiveshell import InteractiveShell
import pandas as pd

sys.path.insert(0, os.path.abspath('../..'))
sys.path.insert(0, os.path.abspath('../../gscapl/'))

import weather as weather

InteractiveShell.ast_node_interactivity = 'all'

### Get a daily weather report by latitude and longitude

In [2]:
now = dt.datetime.now()
today = dt.datetime(year=now.year, month=now.month, day=now.day); today

report = weather.weather_report((47.6062, 122.3321, today))
pd.DataFrame(report.get('report')).head()

datetime.datetime(2019, 1, 15, 0, 0)

,cloud_cover_IQR,cloud_cover_mean,cloud_cover_median,cloud_cover_std,date,dew_point_IQR,dew_point_mean,dew_point_median,dew_point_std,humidity_IQR,...,humidity_median,humidity_std,lat,lon,precip_sum,temp_IQR,temp_mean,temp_med,temp_std,zipcode
0,0.42,0.32,0.2,0.33,2019-01-15,7.28,-12.29,-10.39,6.06,0.22,...,0.39,0.12,47.6062,122.3321,0.0,3.78,6.78,6.01,3.46,4631


If you notice above, report comes back from the weather module as a dictionary. It contains three attributes: hits, misses, and the report itself. `hits` tells you how many times you were able to retrieve a request from the cache while `misses` tells you how many API calls were made. This is extremely valuable for tracking expenses. You can view them as shown in the next cell.

In [3]:
report['hits']
report['misses']

1

0

### Or get a report by zipcode

In [4]:
report = weather.weather_report((today, 98110))
pd.DataFrame(report.get('report')).head()

,cloud_cover_IQR,cloud_cover_mean,cloud_cover_median,cloud_cover_std,date,dew_point_IQR,dew_point_mean,dew_point_median,dew_point_std,humidity_IQR,...,humidity_median,humidity_std,lat,lon,precip_sum,temp_IQR,temp_mean,temp_med,temp_std,zipcode
0,0.53,0.45,0.49,0.29,2019-01-15,2.71,31.93,31.73,1.64,0.12,...,0.75,0.08,47.64697,-122.53456,0.0,6.43,38.8,38.59,4.09,98110


### If you have a series of requests to make, just include a list of a requests. You can set `n_jobs` to use as many cores as you want. 
For example, the following request makes four simultaneous calls to Dark Sky retrieving daily weather summaries by zipcode or lat-lon pairs, and DateTimes in any order. The only order that must be maintained is that the latitude must come before the longitude.

In [5]:
requests = [
    (today, 47579),
    (47579, today-dt.timedelta(days=1)),
    (38.11, -86.92, today-dt.timedelta(days=2)),
    (today-dt.timedelta(days=3), 38.11, -86.92),
    (38.11, today-dt.timedelta(days=4), -86.92),
]

report = weather.weather_report(requests)

pd.DataFrame(report.get('report')).head()
print(f'hits: {report["hits"]}, misses: {report["misses"]}')

,cloud_cover_IQR,cloud_cover_mean,cloud_cover_median,cloud_cover_std,date,dew_point_IQR,dew_point_mean,dew_point_median,dew_point_std,humidity_IQR,...,humidity_median,humidity_std,lat,lon,precip_sum,temp_IQR,temp_mean,temp_med,temp_std,zipcode
0,0.01,0.93,1.00,0.20,2019-01-15,3.24,29.49,29.26,2.32,0.11,...,0.87,0.06,38.11094,-86.91513,0.00,6.00,33.26,33.82,3.59,47579
1,0.00,0.94,1.00,0.17,2019-01-14,1.35,27.58,27.75,0.82,0.12,...,0.83,0.07,38.11094,-86.91513,0.00,1.95,32.94,32.11,1.65,47579
2,0.00,1.00,1.00,0.00,2019-01-13,3.73,34.20,34.27,2.37,0.04,...,0.94,0.04,38.11000,-86.92000,0.03,1.55,35.97,35.70,1.55,47579
3,0.00,0.99,1.00,0.05,2019-01-12,2.86,33.07,33.64,2.01,0.07,...,0.93,0.04,38.11000,-86.92000,0.12,4.50,35.41,36.71,2.55,47579
4,0.97,0.60,0.87,0.44,2019-01-11,3.61,21.15,20.30,3.73,0.26,...,0.79,0.14,38.11000,-86.92000,0.00,12.44,28.86,31.06,5.75,47579


hits: 5, misses: 0


### The default summarization is at the daily level as shown above. To return the results at they hourly level set `summarize = 'none'`

In [7]:
report = weather.weather_report(requests, summarize='none')
report = pd.DataFrame(report.get('report'))
report.head(50)

,apparentTemperature,cloudCover,cloudCoverError,date,dewPoint,hour,humidity,icon,lat,lon,...,temperatureError,time,uvIndex,visibility,windBearing,windBearingError,windGust,windSpeed,windSpeedError,zipcode
0,27.65,1.00,None,2019-01-15,26.86,0,0.97,cloudy,38.11094,-86.91513,...,None,00:00:00,0.0,7.96,NaN,None,0.39,0.00,None,47579
1,28.81,1.00,None,2019-01-15,27.48,1,0.95,cloudy,38.11094,-86.91513,...,None,01:00:00,0.0,8.81,NaN,None,0.00,0.00,None,47579
2,28.81,1.00,None,2019-01-15,26.86,2,0.92,cloudy,38.11094,-86.91513,...,None,02:00:00,0.0,7.89,180.0,None,0.77,0.39,None,47579
3,29.08,1.00,None,2019-01-15,27.34,3,0.93,cloudy,38.11094,-86.91513,...,None,03:00:00,0.0,7.33,229.0,None,2.48,2.47,None,47579
4,30.59,1.00,None,2019-01-15,28.33,4,0.91,cloudy,38.11094,-86.91513,...,None,04:00:00,0.0,8.30,259.0,None,1.42,1.29,None,47579
5,25.17,1.00,None,2019-01-15,28.64,5,0.92,cloudy,38.11094,-86.91513,...,None,05:00:00,0.0,8.11,224.0,None,5.57,5.28,None,47579
6,22.84,1.00,None,2019-01-15,28.64,6,0.92,cloudy,38.11094,-86.91513,...,None,06:00:00,0.0,8.55,239.0,None,9.24,8.48,None,47579
7,24.77,1.00,None,2019-01-15,29.26,7,0.93,cloudy,38.11094,-86.91513,...,None,07:00:00,1.0,8.55,207.0,None,7.07,6.45,None,47579
8,26.41,1.00,None,2019-01-15,29.40,8,0.90,cloudy,38.11094,-86.91513,...,None,08:00:00,1.0,8.55,214.0,None,6.74,5.59,None,47579
9,26.71,1.00,None,2019-01-15,28.65,9,0.84,cloudy,38.11094,-86.91513,...,None,09:00:00,2.0,8.59,215.0,None,6.85,6.82,None,47579
